In [1]:
import numpy as np
import numpy.linalg as la
# import scipy.sparse as sparse

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import timeit
import numba
from numba import jit

from iteration_methods import*
from basic import*

In [2]:
#output settings
np.set_printoptions(precision=2)
%matplotlib qt 

In [3]:
def SOR_polar(matrix, source, grid, init_guess=None, boundary=((0, 0), (0,0)), tolerance=1.e-8, itermax=1000, omega=1.5):

    M, f, u0, B, tol, kmax = matrix, source, init_guess, boundary, tolerance, itermax

    r, th = grid

    Nr, Nth = r.shape[1]-1, th.shape[0]-1

    dr, dth = (r[0, -1] - r[0, 0])/Nr, (th[-1, 0] - th[0, 0])/Nth

    if init_guess is None:

        u0 = np.ones_like(f)

    #assign dirichlet boundary conditions
    u0[:, 0] = B[0][0]                               
    u0[:, -1] = B[0][1]
    u0[0, :] = B[1][0]
    u0[-1, :] = B[1][1]
    
    u = u0
    k = 0
    rel_diff = tol + 1
    conv_hist = []


    #iteration loop using nested loops.
    while  k < kmax and rel_diff > tol:

        u_next = u.copy()

        for j in range(1, Nth):
            for i in range(1, Nr):
                
                u_next[j, i] = (1-omega)*u[j,i] + omega*((u_next[j, i-1] + u[j, i+1])*(r[i]*dth**2(dr/2+1)) + 
                                (u_next[j-1, i] + u[j+1, i])*dr**2 - f[j,i]*dr**2*dth**2*r[i]**2)/(2*(dr**2+r[i]**2*dth**2))
        
        

        rel_diff = la.norm(u_next-u)/la.norm(u)
        
        conv_hist.append(rel_diff)
        
        u = u_next
        k += 1

    return u, k, rel_diff, conv_hist

In [4]:
#Setup the parameters for the solver. Functions grid(), source() are in the basic.py module

R, TH, r, th, dr, dth = polar_grid(th_I=-np.pi, th_F=np.pi)                     #The grid of the problem

f = source(r, th)                                #source term

u_guess = np.cos(np.pi*th)*np.cos(np.pi*th)       #initial guess

boundary = ((0,1), (R, R))                      #dirichlet boundary conditions


iterations = 20000                              #max number of iteration
tolerance = 1.e-6                               #desired tolerance

omega_opt = 2/(1+np.sin(np.pi*max(dr, dth)))     #relaxation parameter for SOR method

u_an = u_analytical(r, th)                       #analytical solution for comparison

In [5]:
SOR = jit(nopython=True)(SOR)

In [6]:
#Calculate the results, time them and print output

start = timeit.default_timer()

u, k, rel_diff, conv_hist = SOR(np.eye(99), f, (r, th), init_guess=u_guess, boundary=boundary,                                                                         tolerance=tolerance, itermax=iterations, omega=omega_opt)

elapsed = timeit.default_timer() - start

error_to_an = la.norm(u - u_an, 2)/la.norm(u_an, 2)

print('SOR solver \nNumber of iterations: {:} \nLast Relative difference: {:1.3E}' 
      '\nError to analytical: {:1.3E}s \nElapsed time: {:1.2E}s '.format(k, rel_diff, error_to_an, elapsed))
print('-'*50)

SOR solver 
Number of iterations: 1176 
Last Relative difference: 9.949E-07
Error to analytical: 1.049E-01s 
Elapsed time: 2.29E+00s 
--------------------------------------------------


In [7]:
#Plot results. Function plot is in basic.py module.
plot(r*np.cos(th), r*np.sin(th), u, u_an=u_an, conv_hist=conv_hist)